In [1]:
import pandas as pd

from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))

In [2]:
mf = pd.read_csv('mf_buildstock.csv', index_col='Building')

In [3]:
dirs = ['North', 'Northeast', 'East', 'Southeast', 'South', 'Southwest', 'West', 'Northwest']
inv_dirs = ['South', 'Southwest', 'West', 'Northwest', 'North', 'Northeast', 'East', 'Southeast']
orient_dict = dict(zip(dirs, inv_dirs))

In [5]:
for idx in mf.index:
    bldg = mf.loc[idx]
    units = int(bldg['Geometry Building Number Units MF'])
    floors = int(bldg['Geometry Building Floors'])
    num_units_per_floor = units/floors

    bstock = pd.DataFrame(columns= mf.columns, index=range(int(1),units+1))
    bstock.index.name = 'Building'
    print("ID: ", idx)
    print("floors: ", floors)
    print("units: ", units)

    front_orient = bldg['Orientation']
    rear_orient = front_orient
    num_middle_z = floors - 2
    if num_units_per_floor%2 != 0: #no rear
        cor_val = 'Single Exterior Front'
        has_rear_units = False
        num_middle_x = int(num_units_per_floor - 2)
    else: #rear units
        cor_val = 'Double-Loaded Interior'
        has_rear_units = True
        rear_orient = orient_dict[bldg['Orientation']]
        num_middle_x = int((num_units_per_floor/2) - 2)   
    if num_middle_x < 0:
        num_middle_x = 0

    if has_rear_units:
        num_units_front = int(units/2)
        units_add = int(num_units_per_floor/2)
    else:
        num_units_front = int(units)
        units_add = int(num_units_per_floor)

    #Initial horizontal locations
    h_left = [1]
    if num_middle_x > 0:
        h_mid = list(range(2, 2+int(num_middle_x)))
    else:
        h_mid = []
    if num_units_front/floors > 1:
        h_right = [num_middle_x + 2]
    else:
        h_right = []
    #Define horizontal locatons by floor
    for f in range(2,floors+1):
        h_left += [h_left[-1] + units_add]
        h_mid += list(map(lambda x: x+units_add, h_mid[-num_middle_x:]))
        h_right += list(map(lambda x: x+units_add, h_right[-1:]))

    #LEVELS
    #..bottom
    if num_units_front/floors > 1:
        lv_bottom = list(range(1,2+num_middle_x+1))
    else:
        lv_bottom = [1]
    #..middle
    if num_middle_z > 0:
        lv_middle = list(range(lv_bottom[-1]+1, lv_bottom[-1]+1 + (units_add*num_middle_z)))
    else:
        lv_middle = []
    #..top
    if floors > 1:
        lv_top = list(range(units_add*(floors-1) + 1, units_add*floors +1))
    else:
        lv_top = []

    if has_rear_units:
        h_left += list(map(lambda x:x+num_units_front, h_left))
        h_mid += list(map(lambda x:x+num_units_front, h_mid))
        h_right += list(map(lambda x:x+num_units_front, h_right))
        lv_bottom += list(map(lambda x:x+num_units_front, lv_bottom))
        lv_middle += list(map(lambda x:x+num_units_front, lv_middle))
        lv_top += list(map(lambda x:x+num_units_front, lv_top))
        
    print("lv_bottom = ", lv_bottom)
    print("lv_middle = ", lv_middle)
    print("lv_top = ", lv_top)
    print("h_left = ", h_left)
    print("h_mid = ", h_mid)
    print("h_right = ", h_right)    
    print("____________________")

    floor_unit = 1
    for unit in range(1,units+1):     
        bstock.loc[unit] = bldg
        bstock.loc[unit, 'Corridor'] = cor_val

        if unit in h_left:
            bstock.loc[unit, 'Geometry Single Unit Horizontal Location'] = 'Left'
        elif unit in h_mid:
            bstock.loc[unit, 'Geometry Single Unit Horizontal Location'] = 'Middle'
        elif unit in h_right:
            bstock.loc[unit, 'Geometry Single Unit Horizontal Location'] = 'Right'

        if unit in lv_bottom:
            bstock.loc[unit, 'Geometry Single Unit Level'] = 'Bottom'
        elif unit in lv_middle:
            bstock.loc[unit, 'Geometry Single Unit Level'] = 'Middle'
        elif unit in lv_top:
            bstock.loc[unit, 'Geometry Single Unit Level'] = 'Top'

        if unit <= units/2:
            bstock.loc[unit, 'Orientation'] = front_orient
        else:
            bstock.loc[unit, 'Orientation'] = rear_orient
            
    bstock.to_csv('./mf_buildstock_singles_new/buildstock_' + str(idx) + '.csv')

ID:  1
floors:  2
units:  12
lv_bottom =  [1, 2, 3, 7, 8, 9]
lv_middle =  []
lv_top =  [4, 5, 6, 10, 11, 12]
h_left =  [1, 4, 7, 10]
h_mid =  [2, 5, 8, 11]
h_right =  [3, 6, 9, 12]
____________________
ID:  2
floors:  1
units:  2
lv_bottom =  [1, 2]
lv_middle =  []
lv_top =  []
h_left =  [1, 2]
h_mid =  []
h_right =  []
____________________
ID:  3
floors:  1
units:  2
lv_bottom =  [1, 2]
lv_middle =  []
lv_top =  []
h_left =  [1, 2]
h_mid =  []
h_right =  []
____________________
ID:  4
floors:  2
units:  12
lv_bottom =  [1, 2, 3, 7, 8, 9]
lv_middle =  []
lv_top =  [4, 5, 6, 10, 11, 12]
h_left =  [1, 4, 7, 10]
h_mid =  [2, 5, 8, 11]
h_right =  [3, 6, 9, 12]
____________________
ID:  5
floors:  2
units:  6
lv_bottom =  [1, 2, 3]
lv_middle =  []
lv_top =  [4, 5, 6]
h_left =  [1, 4]
h_mid =  [2, 5]
h_right =  [3, 6]
____________________
ID:  6
floors:  3
units:  6
lv_bottom =  [1, 4]
lv_middle =  [2, 5]
lv_top =  [3, 6]
h_left =  [1, 2, 3, 4, 5, 6]
h_mid =  []
h_right =  []
_______________

ID:  40
floors:  1
units:  12
lv_bottom =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
lv_middle =  []
lv_top =  []
h_left =  [1, 7]
h_mid =  [2, 3, 4, 5, 8, 9, 10, 11]
h_right =  [6, 12]
____________________
ID:  41
floors:  1
units:  2
lv_bottom =  [1, 2]
lv_middle =  []
lv_top =  []
h_left =  [1, 2]
h_mid =  []
h_right =  []
____________________
ID:  42
floors:  1
units:  3
lv_bottom =  [1, 2, 3]
lv_middle =  []
lv_top =  []
h_left =  [1]
h_mid =  [2]
h_right =  [3]
____________________
ID:  43
floors:  1
units:  2
lv_bottom =  [1, 2]
lv_middle =  []
lv_top =  []
h_left =  [1, 2]
h_mid =  []
h_right =  []
____________________
ID:  44
floors:  2
units:  12
lv_bottom =  [1, 2, 3, 7, 8, 9]
lv_middle =  []
lv_top =  [4, 5, 6, 10, 11, 12]
h_left =  [1, 4, 7, 10]
h_mid =  [2, 5, 8, 11]
h_right =  [3, 6, 9, 12]
____________________
ID:  45
floors:  1
units:  6
lv_bottom =  [1, 2, 3, 4, 5, 6]
lv_middle =  []
lv_top =  []
h_left =  [1, 4]
h_mid =  [2, 5]
h_right =  [3, 6]
____________________
ID

ID:  75
floors:  2
units:  36
lv_bottom =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 19, 20, 21, 22, 23, 24, 25, 26, 27]
lv_middle =  []
lv_top =  [10, 11, 12, 13, 14, 15, 16, 17, 18, 28, 29, 30, 31, 32, 33, 34, 35, 36]
h_left =  [1, 10, 19, 28]
h_mid =  [2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 35]
h_right =  [9, 18, 27, 36]
____________________
ID:  76
floors:  3
units:  15
lv_bottom =  [1, 2, 3, 4, 5]
lv_middle =  [6, 7, 8, 9, 10]
lv_top =  [11, 12, 13, 14, 15]
h_left =  [1, 6, 11]
h_mid =  [2, 3, 4, 7, 8, 9, 12, 13, 14]
h_right =  [5, 10, 15]
____________________
ID:  77
floors:  3
units:  6
lv_bottom =  [1, 4]
lv_middle =  [2, 5]
lv_top =  [3, 6]
h_left =  [1, 2, 3, 4, 5, 6]
h_mid =  []
h_right =  []
____________________
ID:  78
floors:  2
units:  16
lv_bottom =  [1, 2, 3, 4, 9, 10, 11, 12]
lv_middle =  []
lv_top =  [5, 6, 7, 8, 13, 14, 15, 16]
h_left =  [1, 5, 9, 13]
h_mid =  [2, 3, 6, 7, 10, 11, 14, 15]
h_right =  [4, 8, 12, 16]
____

In [6]:
bstock.filter(['Geometry Single Unit Level', 'Geometry Single Unit Horizontal Location', 'Orientation'])

,Geometry Single Unit Level,Geometry Single Unit Horizontal Location,Orientation
Building,,,
1,Bottom,Left,Northeast
2,Bottom,Middle,Northeast
3,Bottom,Right,Northeast
4,Middle,Left,Northeast
5,Middle,Middle,Northeast
6,Middle,Right,Northeast
7,Top,Left,Northeast
8,Top,Middle,Northeast
9,Top,Right,Northeast
